# Practice Session 08: Data streams

In this session we will take a large corpus of documents and compute some statistics using data streams methods.

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

Author: <font color="blue">Bernat Quintilla Castellón</font>

E-mail: <font color="blue">bernat.quintilla01@estudiant.upf.edu</font>

Date: <font color="blue">The current date here</font>

In [1]:
import io
import nltk
import gzip
import random
import statistics
import secrets
import re
import gzip

# 0. Dataset and how to iterate

The input file contain lines of dialogue of a set of movies from the [Movie Dialog Corpus](https://www.kaggle.com/datasets/Cornell-University/movie-dialog-corpus). We will use the file `movie_lines.tsv` which contains the text of the dialogue, about 3 million words in about 300,000 lines of dialogue.

During this practice, **we will never load this file in memory.**

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

In [2]:
# Leave this code as-is

INPUT_FILE = "movie_lines.tsv.gz"

The function `read_by_words` is a [generator](https://wiki.python.org/moin/Generators), that is, a function that behaves as an iterator. This is a common pattern used in stream processing, and in Python is implemented with the `yield` keyword, instead of `return`.

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

In [3]:
# Leave this code as-is

# Producer in Python that reads a filename by words
def read_by_words(filename, max_words=-1, report_every=-1):
    
    # Open the input file
    with gzip.open(INPUT_FILE, "rt", encoding='utf8') as file:
        
        # Initialize counter of words to stop at max_words
        counter = 0
    
        # Regular expression to identify words having 3 letters or more and beginning with a-z
        word_expr = re.compile('^[a-z]{2,}$', re.IGNORECASE)

        # Iterate through lines in the file
        for line in file:
            
            elements = line.split("\t")
            
            text = ""
            if len(elements) >= 5:
                text = elements[4].strip()
                                        
            if counter > max_words and max_words != -1:
                break
                
            for word in nltk.word_tokenize(text):
                          
                if word_expr.match(word):
                    counter += 1
                    
                    # Report
                    if (report_every != -1) and (counter % report_every == 0):
                        if max_words == -1:
                            print("- Read %d words so far" % (counter))
                        else:
                            print("- Read %d/%d words so far" % (counter, max_words))

                    # Produce the word in lowercase
                    yield word.lower()

We will do a first pass over the data. Here we will read only the first 300K words. Try with a larger limit if your computer is fast, with a lower limit if your computer is slow. Find something that makes one pass take about 30 seconds and use it for development.

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

In [6]:
# Leave this code as-is

# Iterate through the file
for word in read_by_words(INPUT_FILE, max_words=300000, report_every=100000):
    # Prints 1/10000 of words
    if random.random() < 0.0001:
        print("Current word '%s'" % (word)) 

Current word 'nakedness'
Current word 'women'
Current word 'die'
Current word 'simon'
Current word 'your'
Current word 'aware'
Current word 'that'
Current word 'said'
- Read 100000/300000 words so far
Current word 'do'
Current word 'before'
Current word 'should'
Current word 'did'
Current word 'my'
Current word 'out'
Current word 'he'
Current word 'other'
Current word 'feels'
Current word 'and'
Current word 'whole'
Current word 'it'
- Read 200000/300000 words so far
Current word 'on'
Current word 'things'
Current word 'grant'
Current word 'fuck'
Current word 'you'
Current word 'gusta'
Current word 'but'
Current word 'so'
- Read 300000/300000 words so far


In [5]:
# Run this if above gives an error about 'punkt'
#nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\berna\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

# 1. Determine approximately the top-10 words

Instead of loading the entire dataset in main memory, we will use reservoir sampling to determine approximately the top-10 words.

**Reservoir sampling**: In reservoir sampling, if we have a reservoir of size S:

* We store the first S elements of the stream
* When the n<sup>th</sup> element arrives (let's call it X<sub>n</sub>):
   * With probability 1 - s/n, we ignore this element.
   * With probability s/n, we:
      * Discard a random element from the reservoir
      * Add element X<sub>n</sub> to the reservoir (calling *add_to_reservoir*)
      
<font size="-1" color="gray">(Remove this cell when delivering.)</font>

Implement a function `add_reservoir(reservoir, item, max_size)` that adds an item to the reservoir, maintaining its size. If the reservoir is already of size *max_size*, a random item is selected and evicted *before* adding the item. It is important to evict an old item *before* adding the new item. Use the following skeleton:

```python
def add_to_reservoir(reservoir, item, max_reservoir_size):
    # YOUR CODE HERE
    assert(len(reservoir) <= max_reservoir_size)
```

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

<font size="+1" color="red">Replace this cell with your code for "add_reservoir"</font>

In [9]:
def add_to_reservoir(reservoir, item, max_reservoir_size):
    # YOUR CODE HERE
    if len(reservoir) < max_reservoir_size:
        reservoir.append(item)
    else:
        item_to_remove = random.choice(reservoir)
        reservoir.remove(item_to_remove)
        reservoir.append(item)
    assert(len(reservoir) <= max_reservoir_size)

Create a function to iterate through the file using the reservoir sampling method seen in class. In this function you will decide, for every item, whether to call *add_to_reservoir* or to ignore the item.

You can use the following skeleton:

```python
def reservoir_sampling(filename, reservoir_size, max_words=-1, report_every=-1):
    reservoir = []
    
    words_read = 0
    
    for word in read_by_words(filename, max_words=max_words, report_every=report_every):
    
            # YOUR CODE HERE

    return (words_read, reservoir)
```

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

<font size="+1" color="red">Replace this cell with your code for "reservoir_sampling"</font>

In [11]:
def reservoir_sampling(filename, reservoir_size, max_words=-1, report_every=-1):
    reservoir = []
    words_read = 0

    for word in read_by_words(filename, max_words=max_words, report_every=report_every):
        words_read += 1

        #Reservoir sampling
        if len(reservoir) < reservoir_size:
            #If reservoir is not full add word directly
            reservoir.append(word)
        else:
            #Reservoir is full
            #Probability of replacing an existing word is reservoir_size/words_read
            replace_probability = reservoir_size / words_read

            if random.random() < replace_probability:
                add_to_reservoir(reservoir, word, reservoir_size)

    return words_read, reservoir

Test your function using the following code:

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

In [12]:
# Leave this code as-is

reservoir_size = 1500
(items_seen, reservoir) = reservoir_sampling(INPUT_FILE, reservoir_size, max_words=1000000, report_every=100000)

print("Number of items seen    : %d" % items_seen)
print("Number of items sampled : %d" % len(reservoir) )

- Read 100000/1000000 words so far
- Read 200000/1000000 words so far
- Read 300000/1000000 words so far
- Read 400000/1000000 words so far
- Read 500000/1000000 words so far
- Read 600000/1000000 words so far
- Read 700000/1000000 words so far
- Read 800000/1000000 words so far
- Read 900000/1000000 words so far
- Read 1000000/1000000 words so far
Number of items seen    : 1000023
Number of items sampled : 1500


The reservoir contains repeated items. You can compute the absolute frequencies of the top 10 using the following code.

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

In [13]:
# Leave this code as-is

freq = {}
for item in reservoir:
    freq[item] = reservoir.count(item)

most_frequent_items = sorted([(frequency, word) for word, frequency in freq.items()], reverse=True)[:10]
for absolute_frequency, word in most_frequent_items:
    print("%d %s" % (absolute_frequency, word))

78 you
54 the
41 to
35 it
28 me
27 do
26 and
23 that
19 for
18 of


Write code to compute the 15 most frequent items in the reservoir and their relative frequencies, as percentages.

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

<font size="+1" color="red">Replace this cell with your code to print the top items and their relative frequencies</font>

In [14]:
#Compute total count of items in the reservoir
total_items = len(reservoir)

#Compute and print top 15 most frequent items and their relative frequencies
top_items = sorted([(frequency, word) for word, frequency in freq.items()], reverse=True)[:15]

for absolute_frequency, word in top_items:
    relative_frequency = (absolute_frequency / total_items) * 100
    print("%d %s (%.2f%%)" % (absolute_frequency, word, relative_frequency))


78 you (5.20%)
54 the (3.60%)
41 to (2.73%)
35 it (2.33%)
28 me (1.87%)
27 do (1.80%)
26 and (1.73%)
23 that (1.53%)
19 for (1.27%)
18 of (1.20%)
17 we (1.13%)
16 on (1.07%)
15 is (1.00%)
15 he (1.00%)
13 what (0.87%)


If you see an item C times in the reservoir, you can estimate the item appears *C x dataset_size / reservoir_size* times in the entire dataset (*dataset_size* is the size of the entire dataset). 

For various sizes of the reservoir, e.g., 50, 100, 500, ..., list the top-5 words and your estimate of their frequency in the entire dataset.
 
<font size="-1" color="gray">(Remove this cell when delivering.)</font>

<font size="+1" color="red">Increase the max limit of words so that one pass takes no more than 5 minutes to be completed. Replace this cell with your code to try different reservoir sizes. In each case, print your estimate for the relative and absolute frequency of the words in the entire dataset.</font>

In [15]:
#Function to estimate the frequency in the entire dataset
def estimate_frequency(absolute_frequency, dataset_size, reservoir_size):
    return (absolute_frequency * dataset_size) / reservoir_size

reservoir_sizes = [50, 100, 500, 1000, 5000]

#Iterate over different reservoir sizes
for reservoir_size in reservoir_sizes:
    print(f"\nReservoir Size: {reservoir_size}\n{'=' * 40}")

    #Perform reservoir sampling
    words_read, reservoir = reservoir_sampling(INPUT_FILE, reservoir_size, max_words=300000, report_every=100000)

    #Compute total count of items in entire dataset
    total_dataset_size = words_read * (300000 / words_read)

    #Compute and print top 5 most frequent items and their estimates
    top_items = sorted([(frequency, word) for word, frequency in freq.items()], reverse=True)[:5]

    for absolute_frequency, word in top_items:
        relative_frequency = (absolute_frequency / words_read) * 100
        estimated_frequency = estimate_frequency(absolute_frequency, total_dataset_size, reservoir_size)

        print(f"{word}:")
        print(f"  Absolute Frequency: {absolute_frequency}")
        print(f"  Relative Frequency: {relative_frequency:.2f}%")
        print(f"  Estimated Frequency in Entire Dataset: {estimated_frequency:.2f}\n")


Reservoir Size: 50
- Read 100000/300000 words so far
- Read 200000/300000 words so far
- Read 300000/300000 words so far
you:
  Absolute Frequency: 78
  Relative Frequency: 0.03%
  Estimated Frequency in Entire Dataset: 468000.00

the:
  Absolute Frequency: 54
  Relative Frequency: 0.02%
  Estimated Frequency in Entire Dataset: 324000.00

to:
  Absolute Frequency: 41
  Relative Frequency: 0.01%
  Estimated Frequency in Entire Dataset: 246000.00

it:
  Absolute Frequency: 35
  Relative Frequency: 0.01%
  Estimated Frequency in Entire Dataset: 210000.00

me:
  Absolute Frequency: 28
  Relative Frequency: 0.01%
  Estimated Frequency in Entire Dataset: 168000.00


Reservoir Size: 100
- Read 100000/300000 words so far
- Read 200000/300000 words so far
- Read 300000/300000 words so far
you:
  Absolute Frequency: 78
  Relative Frequency: 0.03%
  Estimated Frequency in Entire Dataset: 234000.00

the:
  Absolute Frequency: 54
  Relative Frequency: 0.02%
  Estimated Frequency in Entire Dataset:

Find by trial and error, and include in your report, the minimum reservoir size you need to have somewhat stable results (e.g., the same top-3 words in two consecutive runs of the algorithm).

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

<font size="+1" color="red">Remove the max limit of words and re-run. Replace this cell with a brief commentary indicating what reservoir size you would recommend to use, and your overall conclusions.</font>

# 2. Determine approximately the distinct number of words

We will estimate the number of distinct words without creating a dictionary or hash table, but instead, we will use the Flajolet-Martin probabilistic counting method.

**Flajolet-Martin probabilistic counting**:

* For several passes
   * Create hash funcion h
   * For every element *u* in the stream:
      * Compute hash value *h(u)*
      * Let *r(u)* be the number of trailing zeroes in *h(u)*
      * Maintain *R* as the maximum value of *r(u)* seen so far
   * Add *2<sup>R</sup>* as an estimate for the number of distinct elements *u* seen
* The final estimate is the average or the median of the estimates found in each pass

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

Use this function to count trailing zeroes in the binary representation of a number.

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

In [12]:
# Leave this code as-is

def count_trailing_zeroes(number):
    count = 0
    while number & 1 == 0:
        count += 1
        number = number >> 1
    return count

Use this function to generate a random hash function. Note this generates a function, so you can do `hash_function = random_hash_function()` and then call `hash_function(x)` to compute the hash value of `x`. 

We want to make sure each hash is different, so we will create each hash function with a different [salt](https://en.wikipedia.org/wiki/Salt_(cryptography)), which is an additional input that we will take using a good random string generator from the [secrets](https://docs.python.org/3/library/secrets.html) library.

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

In [13]:
# Leave this code as-is

def random_hash_function():
    # We use a cryptographically safe generator for the salt of our hash function
    salt = secrets.token_bytes(32)
    return lambda string: hash(string + str(salt))

Perform *number_of_passes* passes over the file, reading the entire file on each pass (we don't use the reservoir in this part). In each pass, create a new hash function and use it to hash userids. Keep the maximum number of trailing zeroes seen in the hash value of a userid. 

```python
number_of_passes = 5
estimates = []

for i in range(number_of_passes):
    # YOUR_CODE_HERE: read the file and generate an estimate
    
    estimates.append(estimate)
    print("Estimate on pass %d: %d distinct words" % (i+1, estimate))
```

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

<font size="+1" color="red">Replace this cell with your code to perform the requested number of passes.</font>

In [16]:
# Leave this code as-is

print("* Average of estimates: %.1f" % statistics.mean(estimates))
print("* Median  of estimates: %.1f" % statistics.median(estimates))

* Average of estimates: 62259.2
* Median  of estimates: 16384.0


Compute the median of estimates obtained in 3 separate runs of your algorithm; each run should do 10 passes over the file. 

Increase the numbe of passes to 20 and perform 3 separate runs.

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

<font size="+1" color="red">Remove the limit of max words, or set to a high number, but notice that you do no need to use more than one hour of computer processing time, and perform the 10 passes. Replace this cell with the results you obtained in each pass, and whether the average or the median seem more appropriate for this probabilistic counting.</font>

# DELIVER (individually)

Remember to read the section on "delivering your code" in the [course evaluation guidelines](https://github.com/chatox/data-mining-course/blob/master/upf/upf-evaluation.md).

Deliver a zip file containing:

* This notebook

## Extra points available

For more learning and extra points, notice that the number of **distinct** words in a corpus, as a function of the **total** number of words in the corpus, follows an empirical law known as [Heap's Law](https://en.wikipedia.org/wiki/Heaps%27_law).

Repeat the probabilistic counting experiment for various values of `max_word` and plot the total number of words read versus the number of distinct words (remember to label axes). Check if it follows Heap's law.

Please note that using probabilistic counting means a substantial amount of noise will be introduced and perhaps the Heap's law will not be clear in your plot.

**Note:** if you go for the extra points, add ``<font size="+2" color="blue">Additional results: Heap's law</font>`` at the top of your notebook. 

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

<font size="+2" color="#003300">I hereby declare that, except for the code provided by the course instructors, all of my code, report, and figures were produced by myself.</font>